<a href="https://colab.research.google.com/github/yukinaga/image_generation/blob/main/section_3/01_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オートエンコーダの実装
VAEの実装に入る前に、通常のオートエンコーダを実装しましょう。   
Encoderで中間層に画像を圧縮した後に、Decoderで元の画像を再構築します。  

## 手書き文字画像
今回は、オートエンコーダにより手書き文字画像を圧縮、復元します。  
scikit-learnから、8×8の手書き数字の画像データを読み込んで表示します。  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

digits_data = datasets.load_digits()

n_img = 10  # 表示する画像の数
plt.figure(figsize=(10, 4))
for i in range(n_img):
    # 入力画像
    ax = plt.subplot(2, 5, i+1)
    plt.imshow(digits_data.data[i].reshape(8, 8), cmap="Greys_r")
    ax.get_xaxis().set_visible(False)  # 軸を非表示に
    ax.get_yaxis().set_visible(False)
plt.show()

print("データの形状:", digits_data.data.shape)
print("ラベル:", digits_data.target[:n_img])

## 各設定
画像の幅と高さが8ピクセルなので、入力層には8×8=64のニューロンが必要になります。  
また、出力が入力を再現するように学習するので、出力層のニューロン数は入力層と同じになります。  
中間層にはこれらよりも少ないニューロンを配置します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

import torch
from torch.utils.data import DataLoader

# -- 各設定値 --
img_size = 8  # 画像の高さと幅
n_in_out = img_size * img_size  # 入出力層のニューロン数
n_mid = 16  # 中間層のニューロン数

eta = 0.01  # 学習係数
epochs = 100
batch_size = 16
interval = 10  # 経過の表示間隔

# -- 訓練データ --
digits_data = datasets.load_digits()
x_train = np.asarray(digits_data.data)
x_train /= 16  # 0-1の範囲に

x_train = torch.tensor(x_train, dtype=torch.float)
train_dataset = torch.utils.data.TensorDataset(x_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## モデルの構築
PyTorchよりオートエンコーダのモデルを構築します。  
Encoder、Decoderの順に層を重ねます。  
入力の値は0から1の範囲なのですが、出力の範囲をこれに合わせる必要があります。  
従って、出力層の活性化関数には出力範囲が0から1に収まるシグモイド関数を使います。  

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Linear(n_in_out, n_mid)  # Encoder
        self.decoder = nn.Linear(n_mid, n_in_out)  # Decoder

    def forward(self, x):
        x = x.view(-1, n_in_out)  # バッチサイズ×入力の数
        x = F.relu(self.encoder(x))
        x = F.sigmoid(self.decoder(x))
        return x

    def encode(self, x):
        x = x.view(-1, n_in_out)  # バッチサイズ×入力の数
        x = F.relu(self.encoder(x))
        return x

autoencoder = Autoencoder()
autoencoder.cuda()  # GPU対応
print(autoencoder)

## 学習
構築したオートエンコーダのモデルを使って、学習を行います。  
入力を再現するように学習するので、正解は入力そのものになります。

In [ ]:
from torch import optim

# 二乗和誤差
loss_fnc = nn.MSELoss()

# Adam
optimizer = optim.Adam(autoencoder.parameters())

# 損失のログ
record_loss_train = []

# 学習
for i in range(epochs):
    autoencoder.train()  # 訓練モード
    loss_train = 0
    for j, (x,) in enumerate(train_loader):  # ミニバッチ（x,）を取り出す
        x = x.cuda()  # GPU対応
        y = autoencoder(x)
        loss = loss_fnc(y, x)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)

    if i%interval == 0:
        print("Epoch:", i, "Loss_Train:", loss_train)

## 誤差の推移
記録された誤差の、推移を確認します。

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

誤差が滑らかに減少している様子が確認できます。

## 生成された画像の表示
画像が適切に再構築されているかどうか、中間層がどのような状態にあるのかを確認します。  
入力画像と、再構築された画像を並べて表示します。  
また、エンコーダーの出力も4×4の画像として表示します。  

In [ ]:
n_img = batch_size  # 表示する画像の数
x = next(iter(train_loader))[0]
x = x.cuda()

autoencoder.eval()  # 評価モード
m = autoencoder.encode(x)  # 中間層の状態
y = autoencoder(x)

# NumPyの配列に変換
x_sample = x.cpu().detach().numpy()
m_sample = m.cpu().detach().numpy()
y_sample = y.cpu().detach().numpy()

plt.figure(figsize=(n_img, 3))
for i in range(n_img):
    # 入力画像
    ax = plt.subplot(3, n_img, i+1)
    plt.imshow(x_sample[i].reshape(img_size, -1).tolist(), cmap="Greys_r")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # 中間層の出力
    ax = plt.subplot(3, n_img, i+1+n_img)
    plt.imshow(m_sample[i].reshape(4, -1).tolist(), cmap="Greys_r")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # 出力画像
    ax = plt.subplot(3, n_img, i+1+2*n_img)
    plt.imshow(y_sample[i].reshape(img_size, -1).tolist(), cmap="Greys_r")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

出力は入力をある程度再現した画像になっており、中間層は数字画像ごとに異なる状態となっています。  
64ピクセルの画像を特徴付ける情報を、16の状態に圧縮できたことになります。  
しかしながら、中間層の状態と出力画像の対応関係を直感的に把握したり、中間層の状態を調整して出力画像を変化させることは難しそうです。  


## 演習
中間層のニューロン数が少なくなると、次第にうまく画像を再構築できなくなります。  
実際にニューロン数を少なくし、再構築ができなくなることを確認しましょう。